In [1]:
from weaviate import Config
import weaviate
import os

palm_key = "ya29.a0AfB_byBjOgGR3ALAY2DVA7okO68x5eilqu4FXPaGoPy5XgpLNveGJpzQiraz5gb0D-V3fqYs6lQzEKLLRCOv7ZD9Xoygya9KUHp4o978xolIG-iqVxgw6cUtOgjZtWSZh2MwzU_BfJp5llrIjwX_0tLFyekYVo0mcw3BHZMOR70aCgYKAfoSARISFQGOcNnCy1leP3msdS0urC7O2PLejw0178"

client = weaviate.Client(
    "http://localhost:8080",
    additional_config=Config(grpc_port_experimental=50051),
    additional_headers={
        "X-OpenAI-Api-Key": os.environ["OPENAI_APIKEY"],
        "X-HuggingFace-Api-Key": os.environ["HUGGINGFACE_APIKEY"],
        "X-Palm-Api-Key": palm_key,
    }
)

In [3]:
import weaviate.weaviate_classes as wvc

articles = client.collection.create(
    config=wvc.CollectionConfig(
        name="TestArticle",
        properties=[
            wvc.Property(
                name="title",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="body",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="url",
                data_type=wvc.DataType.TEXT,
                tokenization=wvc.Tokenization.FIELD
            ),
        ],
        vectorizer_config=wvc.Text2VecHuggingFaceConfig(options=wvc.Text2VecHuggingFaceConfigOptions(wait_for_model=True)),
    )
)

In [4]:
uuid = articles.data.insert(
    {
        "title": "Something something dark side",
        "body": "A long long time ago, in a galaxy far, far away...",
        "url": "http://www.starwars.com"
    }
)

In [5]:
uuid

UUID('c5ebdc68-5a61-4782-9ef3-99019c8aee2e')

In [9]:
resp = articles.data.get(metadata=wvc.GetObjectsMetadata(vector=True))
len(resp[0].metadata.vector)

768

In [10]:
client.schema.delete_class("TestArticle")

import weaviate.weaviate_classes as wvc

articles = client.collection.create(
    config=wvc.CollectionConfig(
        name="TestArticle",
        properties=[
            wvc.Property(
                name="title",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="body",
                data_type=wvc.DataType.TEXT,
            ),
            wvc.Property(
                name="url",
                data_type=wvc.DataType.TEXT,
                tokenization=wvc.Tokenization.FIELD
            ),
        ],
        vectorizer_config=wvc.Text2VecPalmConfig(project_id="devrel-projects"),
    )
)

In [11]:
uuid = articles.data.insert(
    {
        "title": "Something something dark side",
        "body": "A long long time ago, in a galaxy far, far away...",
        "url": "http://www.starwars.com"
    }
)

In [12]:
resp = articles.data.get(metadata=wvc.GetObjectsMetadata(vector=True))
len(resp[0].metadata.vector)

768

In [13]:
uuid

UUID('7776b088-8f55-4122-a239-9a1934cdb1c3')

In [15]:
articles.config.get()

_CollectionConfig(name='TestArticle', description=None, inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False), properties=[_Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='title', tokenization=<Tokenization.WORD: 'word'>), _Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='body', tokenization=<Tokenization.WORD: 'word'>), _Property(data_type=<DataType.TEXT: 'text'>, description=None, index_filterable=True, index_searchable=True, name='url', tokenization=<Tokenization.FIELD: 'field'>)], replication_factor=_ReplicationFactor(factor=1), sharding_config=_ShardingConfig(virtual_per_physical=128, desired_count=1, actual_count=1, desired_virtual_count=128, actual_vi